# Morabito 2021

In [1]:
import scanpy as sc
import os,sys,glob
import pandas as pd
import numpy as np
import anndata as ad

In [2]:
m_morabito = pd.read_csv('/home/sonic/scData/Morabito2021/GSE174367_snRNA-seq_cell_meta.csv.gz', sep=',', index_col=0)
m_morabito.rename(columns={ 'SampleID':'sampleID', 'Cell.Type': 'cluster_main', 'cluster':'cluster_original', 'Batch':'batch'}, inplace=True)

In [3]:
m_morabito.groupby(['cluster_original', 'cluster_main']).size().reset_index(name='counts')

,cluster_original,cluster_main,counts
0,ASC1,ASC,2360
1,ASC2,ASC,1451
2,ASC3,ASC,506
3,ASC4,ASC,439
4,EX1,EX,2699
5,EX2,EX,2492
6,EX3,EX,731
7,EX4,EX,336
8,EX5,EX,111
9,INH1,INH,2190


In [4]:
# Update the cell type
m_morabito.loc[m_morabito['cluster_original'].str.contains('EX'), 'cluster_main'] = 'Ext'
m_morabito.loc[m_morabito['cluster_original'].str.contains('INH'), 'cluster_main'] = 'IN'
m_morabito.loc[m_morabito['cluster_original'].str.contains('MG'), 'cluster_main'] = 'MG'
m_morabito.loc[m_morabito['cluster_original'].str.contains('ASC'), 'cluster_main'] = 'Astro'
m_morabito.loc[m_morabito['cluster_original'].str.contains('ODC'), 'cluster_main'] = 'OD'
m_morabito.loc[m_morabito['cluster_original'].str.contains('OPC'), 'cluster_main'] = 'OPC'
m_morabito.loc[m_morabito['cluster_original'].str.contains('PER.END'), 'cluster_main'] = 'Others'

In [5]:
# Update age
m_morabito['Age'] = m_morabito['Age'] * 365 + 365

# Add the columns for matching with other studies
m_morabito['Assay'] = 'Unknown'
m_morabito['Race'] = 'Unknown'
m_morabito['PMI'] = 'Unknown'
m_morabito['Brain_Region'] = 'prefrontal cortex'
m_morabito['Stage'] = 'Adult'
m_morabito['Dataset'] = 'Morabito'

m_morabito['Hemisphere'] = 'Unknown'
m_morabito['Assay'] = "10x Chromium 3' v3"
m_morabito['Library'] = "snRNA"



In [6]:
m_morabito

,sampleID,Diagnosis,batch,cluster_main,cluster_original,Age,Sex,PMI,Tangle.Stage,Plaque.Stage,RIN,Assay,Race,Brain_Region,Stage,Dataset,Hemisphere,Library
Barcode,,,,,,,,,,,,,,,,,,
AAACCCAAGAAGCTCG-1,Sample-19,AD,3,OD,ODC9,33215,F,Unknown,Stage 6,Stage C,6.8,10x Chromium 3' v3,Unknown,prefrontal cortex,Adult,Morabito,Unknown,snRNA
AAACCCAAGGATGCGT-1,Sample-19,AD,3,OD,ODC7,33215,F,Unknown,Stage 6,Stage C,6.8,10x Chromium 3' v3,Unknown,prefrontal cortex,Adult,Morabito,Unknown,snRNA
AAACGAAGTTCGAAGG-1,Sample-19,AD,3,OD,ODC1,33215,F,Unknown,Stage 6,Stage C,6.8,10x Chromium 3' v3,Unknown,prefrontal cortex,Adult,Morabito,Unknown,snRNA
AAAGAACGTATTCCGA-1,Sample-19,AD,3,OD,ODC2,33215,F,Unknown,Stage 6,Stage C,6.8,10x Chromium 3' v3,Unknown,prefrontal cortex,Adult,Morabito,Unknown,snRNA
AAAGGATAGGCCTAAG-1,Sample-19,AD,3,OD,ODC9,33215,F,Unknown,Stage 6,Stage C,6.8,10x Chromium 3' v3,Unknown,prefrontal cortex,Adult,Morabito,Unknown,snRNA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGCTTGAGCCATCCG-16,Sample-100,Control,1,Others,PER.END3,29200,M,Unknown,Stage 2,Stage B,8.7,10x Chromium 3' v3,Unknown,prefrontal cortex,Adult,Morabito,Unknown,snRNA
TGCAGATGTTCCATTT-16,Sample-100,Control,1,Others,PER.END3,29200,M,Unknown,Stage 2,Stage B,8.7,10x Chromium 3' v3,Unknown,prefrontal cortex,Adult,Morabito,Unknown,snRNA
TGCTGAACACTATCGA-16,Sample-100,Control,1,Others,PER.END3,29200,M,Unknown,Stage 2,Stage B,8.7,10x Chromium 3' v3,Unknown,prefrontal cortex,Adult,Morabito,Unknown,snRNA


In [23]:
adata = sc.read_10x_h5('/home/sonic/scData/Morabito2021/GSE174367_snRNA-seq_filtered_feature_bc_matrix.h5')
adata.var_names_make_unique()
adata.obs = pd.concat([adata.obs, m_morabito], axis=1)

/home/sonic/miniconda3/envs/scvi-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [24]:
adata = adata[~adata.obs.sampleID.isna()]

In [25]:
adata

View of AnnData object with n_obs × n_vars = 61472 × 58721
    obs: 'sampleID', 'Diagnosis', 'batch', 'cluster_main', 'cluster_original', 'Age', 'Sex', 'PMI', 'Tangle.Stage', 'Plaque.Stage', 'RIN', 'Assay', 'Race', 'Brain_Region', 'Stage', 'Dataset', 'Hemisphere', 'Library'
    var: 'gene_ids', 'feature_types', 'genome'

In [26]:
adata.obs.loc[adata.obs['Diagnosis']=="Control", 'Diagnosis'] = 'normal'

In [27]:
adata.obs.Diagnosis.value_counts()

AD        38676
normal    22796
Name: Diagnosis, dtype: int64

In [28]:
set(list(adata.obs.sampleID))

{'Sample-100',
 'Sample-17',
 'Sample-19',
 'Sample-22',
 'Sample-27',
 'Sample-33',
 'Sample-37',
 'Sample-43',
 'Sample-45',
 'Sample-46',
 'Sample-47',
 'Sample-50',
 'Sample-52',
 'Sample-58',
 'Sample-66',
 'Sample-82',
 'Sample-90',
 'Sample-96'}

In [29]:
# Update index
adata.obs = adata.obs.reset_index().rename(columns={'index': 'new_index'})
adata.obs['new_index'] = adata.obs.sampleID + '_' + adata.obs['new_index'] 
adata.obs = adata.obs.set_index('new_index')

In [30]:
adata.obs

,sampleID,Diagnosis,batch,cluster_main,cluster_original,Age,Sex,PMI,Tangle.Stage,Plaque.Stage,RIN,Assay,Race,Brain_Region,Stage,Dataset,Hemisphere,Library
new_index,,,,,,,,,,,,,,,,,,
Sample-19_AAACCCAAGAAGCTCG-1,Sample-19,AD,3.0,OD,ODC9,33215.0,F,Unknown,Stage 6,Stage C,6.8,10x Chromium 3' v3,Unknown,prefrontal cortex,Adult,Morabito,Unknown,snRNA
Sample-19_AAACCCAAGGATGCGT-1,Sample-19,AD,3.0,OD,ODC7,33215.0,F,Unknown,Stage 6,Stage C,6.8,10x Chromium 3' v3,Unknown,prefrontal cortex,Adult,Morabito,Unknown,snRNA
Sample-19_AAACCCAGTTGCTCAA-1,Sample-19,AD,3.0,MG,MG1,33215.0,F,Unknown,Stage 6,Stage C,6.8,10x Chromium 3' v3,Unknown,prefrontal cortex,Adult,Morabito,Unknown,snRNA
Sample-19_AAACGAAAGAGGCTGT-1,Sample-19,AD,3.0,MG,MG1,33215.0,F,Unknown,Stage 6,Stage C,6.8,10x Chromium 3' v3,Unknown,prefrontal cortex,Adult,Morabito,Unknown,snRNA
Sample-19_AAACGAACAGAGGTTG-1,Sample-19,AD,3.0,MG,MG1,33215.0,F,Unknown,Stage 6,Stage C,6.8,10x Chromium 3' v3,Unknown,prefrontal cortex,Adult,Morabito,Unknown,snRNA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sample-47_TTTGGTTAGATGCCGA-18,Sample-47,AD,3.0,OD,ODC10,33215.0,M,Unknown,Stage 5,Stage B,8.0,10x Chromium 3' v3,Unknown,prefrontal cortex,Adult,Morabito,Unknown,snRNA
Sample-47_TTTGGTTCAGAGCGTA-18,Sample-47,AD,3.0,Astro,ASC1,33215.0,M,Unknown,Stage 5,Stage B,8.0,10x Chromium 3' v3,Unknown,prefrontal cortex,Adult,Morabito,Unknown,snRNA
Sample-47_TTTGTTGCACATACGT-18,Sample-47,AD,3.0,IN,INH4,33215.0,M,Unknown,Stage 5,Stage B,8.0,10x Chromium 3' v3,Unknown,prefrontal cortex,Adult,Morabito,Unknown,snRNA


In [31]:
adata.write('/home/sonic/scData/anndata/Morabito2021.h5ad')